## Author

In [ ]:
'''
Author: Xin Zan 489072703@qq.com
Date: 2025-3-25 14:26:29
'''

## Calibration

In [1]:
import cv2
assert cv2.__version__[0] == '3', 'The fisheye module requires opencv version >= 3.0.0'
import numpy as np
import os
from path import Path
import matplotlib.pyplot as plt

img_dir = Path(f'/home/zanxin/zanxin/calib/290_230816')
images = sorted(img_dir.glob('*.jpg'))
for img in images:
    print(img)


/home/zanxin/zanxin/calib/290_230816/frame_000001.jpg
/home/zanxin/zanxin/calib/290_230816/frame_000002.jpg
/home/zanxin/zanxin/calib/290_230816/frame_000003.jpg
/home/zanxin/zanxin/calib/290_230816/frame_000004.jpg
/home/zanxin/zanxin/calib/290_230816/frame_000005.jpg
/home/zanxin/zanxin/calib/290_230816/frame_000006.jpg
/home/zanxin/zanxin/calib/290_230816/frame_000007.jpg
/home/zanxin/zanxin/calib/290_230816/frame_000008.jpg
/home/zanxin/zanxin/calib/290_230816/frame_000009.jpg
/home/zanxin/zanxin/calib/290_230816/frame_000010.jpg
/home/zanxin/zanxin/calib/290_230816/frame_000011.jpg
/home/zanxin/zanxin/calib/290_230816/frame_000012.jpg
/home/zanxin/zanxin/calib/290_230816/frame_000013.jpg
/home/zanxin/zanxin/calib/290_230816/frame_000014.jpg
/home/zanxin/zanxin/calib/290_230816/frame_000015.jpg
/home/zanxin/zanxin/calib/290_230816/frame_000016.jpg
/home/zanxin/zanxin/calib/290_230816/frame_000017.jpg
/home/zanxin/zanxin/calib/290_230816/frame_000018.jpg
/home/zanxin/zanxin/calib/29

In [ ]:
# CHECKERBOARD = (15, 10) # 
# square_size = 10 # c3vd

CHECKERBOARD = (8, 8) # 9x9 的棋盘格， pattern_size应该为 8x8
square_size = 6 # 6 看棋盘格的尺寸，我的为 9*9*6mm

subpix_criteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)
calibration_flags = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC+cv2.fisheye.CALIB_CHECK_COND+cv2.fisheye.CALIB_FIX_SKEW
objp = np.zeros((1, CHECKERBOARD[0]*CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2) * square_size

_img_shape = None
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

for fname in images:
    img = cv2.imread(fname)
    if _img_shape == None:
        _img_shape = img.shape[:2]
    else:
        assert _img_shape == img.shape[:2], "All images must share the same size."
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    # ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_FAST_CHECK+cv2.CALIB_CB_NORMALIZE_IMAGE)
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_FAST_CHECK+cv2.CALIB_CB_NORMALIZE_IMAGE)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        cv2.cornerSubPix(gray,corners,(3,3),(-1,-1),subpix_criteria)
        imgpoints.append(corners)
        
    #可视化提取的角点
        cv2.drawChessboardCorners(img, CHECKERBOARD, corners, ret)
        plt.figure(figsize=(10, 10))
        # plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(img)
        plt.show()
    
    else:
        print(f'{fname} has not detected corners')
        
N_OK = len(objpoints)
K = np.zeros((3, 3))
D = np.zeros((4, 1))
rvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
tvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]

rms, _, _, _, _ = \
    cv2.fisheye.calibrate(
        objpoints,
        imgpoints,
        gray.shape[::-1],
        K,
        D,
        rvecs,
        tvecs,
        calibration_flags,
        (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6)
    )
print("Found " + str(N_OK) + " valid images for calibration")
print("DIM=" + str(_img_shape[::-1]))
print("K=np.array(" + str(K.tolist()) + ")")
print("D=np.array(" + str(D.tolist()) + ")")
print(f'rms is: {rms}')

# # 计算每张图的重投影误差
# for i in range(N_OK):
#     projected_points, _ = cv2.fisheye.projectPoints(objpoints[i].reshape(-1,1,3), rvecs[i], tvecs[i], K, D)
    
#     error = imgpoints[i] - projected_points
#     rms_error = np.sqrt(np.mean(error**2))
#     print(f'Image {images[i]} RMS error: {rms_error}')

## Undistortion

In [1]:
import numpy as np
from tqdm import tqdm
from path import Path
import cv2

# # 260 240904
# DIM=(1162, 1007)
# K=np.array([[715.6165462654658, 0.0, 579.6111529424028], [0.0, 716.3003536681483, 512.0439619738358], [0.0, 0.0, 1.0]])
# D=np.array([[-0.1136918020178839], [-0.00270267936430549], [-0.0848114182515754], [0.13132640795529404]])
# fx, fy = K[0, 0], K[1, 1]
# cx, cy = K[0, 2], K[1, 2]
# # rms is: 0.641217717542028

# # 290 230816
# DIM=(1162, 1007)
# K=np.array([[717.323474888361, 0.0, 576.588439310983], [0.0, 717.1064552631971, 510.42880651166683], [0.0, 0.0, 1.0]])
# D=np.array([[-0.10448185927561918], [-0.09679781611180353], [0.26074952843829363], [-0.2652240025401324]])
# # rms is: 0.5726074686684097

# c3vd
DIM=(1350, 1080)
K=np.array([[767.3980383329763, 0.0, 679.0545976908313], [0.0, 767.5170358974989, 543.645636252986], [0.0, 0.0, 1.0]])
D=np.array([[-0.1887138199236522], [-0.00379847687735559], [0.030352421612693584], [-0.012679529136190738]])
# rms is: 0.6742308502882097

In [ ]:
def undistort(img_path, output_path):
    img = cv2.imread(img_path)
    h, w = img.shape[:2]
    
    map1, map2 = cv2.fisheye.initUndistortRectifyMap(K, D, np.eye(3), K, DIM, cv2.CV_16SC2)
    undistorted_img = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
    # cv2.imwrite(f'{output_path}/{img_path.stem:04d}+.png', undistorted_img)
    cv2.imwrite(output_path/img_path.stem+'.png', undistorted_img)


##### 处理C3VD数据集的cecum_t1_a文件夹的rgb文件
# def undistort_t1_a_color(img_path, output_path):
#     img = cv2.imread(img_path)
#     h, w = img.shape[:2]

#     map1, map2 = cv2.fisheye.initUndistortRectifyMap(K, D, np.eye(3), K, DIM, cv2.CV_16SC2)
#     undistorted_img = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
#     cv2.imwrite(f'{output_path}/{int(img_path.stem[:-6]):04d}_color.png', undistorted_img)
#     # cv2.imwrite(output_path/img_path.stem+'.png', undistorted_img)

###### 处理C3VD数据集的深度和光流
def undistort_depth(img_path, output_path):
    img = cv2.imread(img_path, -1) # uint16 65535, uint8 255
    h, w = img.shape[:2]
    
    map1, map2 = cv2.fisheye.initUndistortRectifyMap(K, D, np.eye(3), K, DIM, cv2.CV_16SC2)
    undistorted_img = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
    undistorted_img = 100.0 * undistorted_img.astype(np.float32) / 65535.0
    # cv2.imwrite(output_path/img_path.stem+'.png', undistorted_img)
    (output_path/'gt_npy').makedirs_p()
    np.save(output_path/'gt_npy'/img_path.stem+'.npy', undistorted_img)
    # return undistorted_img

def undistort_flow(img_path, output_path):
    img = cv2.imread(img_path, -1) # uint16 65535, uint8 255
    h, w = img.shape[:2]
    
    map1, map2 = cv2.fisheye.initUndistortRectifyMap(K, D, np.eye(3), K, DIM, cv2.CV_16SC2)
    undistorted_img = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
    undistorted_img = 40.0 * ((undistorted_img.astype(np.float32) - 32768.0) / 65535.0)
    # cv2.imwrite(output_path/img_path.stem+'.png', undistorted_img)
    (output_path/'gt_npy_flow').makedirs_p()
    np.save(output_path/'gt_npy_flow'/img_path.stem+'.npy', undistorted_img)
    # return undistorted_img

In [3]:
distortion_img_path = Path(f'/home/zanxin/zanxin/datasets/241211/crop')
output_path = Path(f'/home/zanxin/zanxin/datasets/241211/undist_fisheye')

scenes = distortion_img_path.listdir()
for scene in scenes:
    output_dir = Path(output_path/scene.stem)
    output_dir.makedirs_p()
    
    images = sorted(scene.glob('*.png'))
    for img in tqdm(images, desc=f'Processing scene {scene.stem}:'):
        undistort(img, output_dir)
    



Processing scene 0034996321:: 100%|██████████| 16/16 [00:01<00:00,  8.47it/s]


In [ ]:
# cfhq190l_10x10mm_checkerboard_images
# c3vd_path = Path(f'/mnt/c3vd_origin')
# output_path = Path(f'/home/zanxin/zanxin/datasets/c3vd_fisheye_opencv')

c3vd_path = Path(f'/root/autodl-tmp/c3vd-dataset')
output_path = Path(f'/root/autodl-fs/c3vd-KB-fisheye')

scenes = c3vd_path.listdir()
scenes = sorted([f for f in scenes if f.stem != 'cfhq190l_10x10mm_checkerboard_images' and f.stem != '.Trash-1000' and f.stem != 'undist_fisheye' and f.stem != 'unzip_all'])


for scene in scenes:
    print(scene.stem)
    
    output_dir = Path(output_path/scene.stem)
    output_dir.makedirs_p()
    
    rgb = sorted(scene.glob('*color.png'))
    depth = sorted(scene.glob('*depth.tiff'))
    flow = sorted(scene.glob('*flow.tiff'))

    # for img in tqdm(rgb, desc=f'Processing scene {scene.stem} rgb:'):
    #     # undistort(img, output_dir)
    #     undistort_t1_a_color(img, output_dir)
    #     # print(img.stem[:-6])
    
    for img in tqdm(depth, desc=f'depth:'):
        undistort_depth(img, output_dir)
        undistort(img, output_dir)

    for img in tqdm(flow, desc=f'flow:'):
        undistort_flow(img, output_dir)
        undistort(img, output_dir)
    

calib_frames
cecum_t1_a
cecum_t1_b
cecum_t2_a
cecum_t2_b
cecum_t2_c
cecum_t3_a
cecum_t4_a
cecum_t4_b
desc_t4_a
sigmoid_t1_a
sigmoid_t2_a
sigmoid_t3_a
sigmoid_t3_b
trans_t1_a
trans_t1_b
trans_t2_a
trans_t2_b
trans_t2_c
trans_t3_a
trans_t3_b
trans_t4_a
trans_t4_b
